{ *** 2. COLLECT THE COMPLEXES DATA FROM THE API ENDPOINTS *** }

In [10]:
import pandas as pd
import psycopg2

#### postgresql connection credentials

# connection = psycopg2.connect(
#    host="localhost",
#    user="postgres",
#    password="vijay45",
#    database="postgres"

# )

# cursor=connection.cursor()

In [11]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=3cx83hgyhyf1YOaeIdPIxYu10bw5H0JSANgpnVSI"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers).json()


complexes_data = response['complexes']


In [12]:
# FETCH DATA FROM API AND CONVERTED INTO DATAFRAME FOR COMPLEXES_DF

complex_id = []
complex_name = []
for value in complexes_data:
    complex_id.append(value['id'])
    complex_name.append(value['name'])

data = {'complex_id': complex_id, 'complex_name': complex_name}
complexes_df = pd.DataFrame(data)

complexes_df.drop_duplicates()

#  FETCH DATA FROM API AND CONVERTED INTO DATAFRAME FOR VENUES_DF


venue_id = []
venue_name = []
city_name = []
country_name = []
country_code = []
timezone = []
complex_id = []

count = []

for value in complexes_data:
    if 'venues' in value:
        for venue in value['venues']:
            venue_id.append(venue['id'])
            venue_name.append(venue['name'])
            city_name.append(venue['city_name'])
            country_name.append(venue['country_name'])
            country_code.append(venue['country_code'])
            timezone.append(venue['timezone'])
            complex_id.append(value['id'])
    else:
        count.append(1)


data = {'venue_id': venue_id, 'venue_name': venue_name, 'city_name':city_name, 'country_name': country_name, 'country_code': country_code, 'timezone': timezone, 'complex_id': complex_id}
venues_df = pd.DataFrame(data)
print(venues_df.shape)
print(sum(count))

(3220, 7)
152


In [14]:

# Establish the database connection

connection = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="vijay45",
    database="postgres"
)
cursor = connection.cursor()

# CREATING SQL TABLE FOR [COMPLEXES]

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Complex (
        complex_id VARCHAR(50) PRIMARY KEY,
        complex_name VARCHAR(100) NOT NULL
    );
""")

# CREATING SQL TABLE FOR [VENUES]

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Venue (
        venue_id VARCHAR(50) PRIMARY KEY,
        venue_name VARCHAR(100) NOT NULL,
        city_name VARCHAR(100) NOT NULL,
        country_name VARCHAR(100) NOT NULL,
        country_code CHAR(3) NOT NULL,
        timezone VARCHAR(100) NOT NULL,
        complex_id VARCHAR(50),
        FOREIGN KEY (complex_id) REFERENCES Complexes(complex_id)
    );
""")


# Insert data into complexes table

for index, row in complexes_df.iterrows():
    cursor.execute('''
    INSERT INTO complex (complex_id, complex_name)
    VALUES (%s, %s);
    ''', (row['complex_id'], row['complex_name']))

# Insert data into venues table

for index, row in venues_df.iterrows():
    cursor.execute('''
    INSERT INTO venue (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
    ''', (row['venue_id'], row['venue_name'], row['city_name'], row['country_name'], row['country_code'], row['timezone'], row['complex_id']))

connection.commit()
connection.close()

print("Data has been inserted into POSTGRESQL!")

Data has been inserted into POSTGRESQL!
